# NB Model Tuning

In this notebook, we will take the folds generated by the OU class in the previous notebook  and try to find the best set of parameters for our Naive Bayes to perform binary classification. 

In [2]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import preprocessing

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

import pickle

import OU


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
save_dir = './data/'

directly read splits results calculated from "SVM Model Tuning.ipynb"

In [4]:
splits = np.load(save_dir + "/splits.npy", allow_pickle=True)

In [5]:
multi_cv_df = pd.read_csv(save_dir + 'df.csv', index_col = 0)
multi_cv_labels = pd.read_csv(save_dir + 'labels.csv', index_col = 0)

# Gridsearch

We want to find the optimal hyperparameters for our XGB by exploring all combinations of possible hyperparameter

In [6]:
params = [{
            'var_smoothing': np.logspace(0, -9, num = 20)
        }]

In [7]:
# Use all cores (n_jobs-1)
gridcv = GridSearchCV(GaussianNB(), params, verbose = 1, cv = list(splits), n_jobs = -1, 
                    scoring = ['precision'], refit = False)

gridcv.fit(multi_cv_df, multi_cv_labels)

Fitting 399 folds for each of 20 candidates, totalling 7980 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

GridSearchCV(cv=[array([RangeIndex(start=0, stop=2000, step=1),
       RangeIndex(start=2000, stop=2100, step=1)], dtype=object),
                 array([RangeIndex(start=2100, stop=4100, step=1),
       RangeIndex(start=4100, stop=4200, step=1)], dtype=object),
                 array([RangeIndex(start=4200, stop=6200, step=1),
       RangeIndex(start=6200, stop=6300, step=1)], dtype=object),
                 array([RangeIndex(start=6300, s...
             param_grid=[{'var_smoothing': array([1.00000000e+00, 3.35981829e-01, 1.12883789e-01, 3.79269019e-02,
       1.27427499e-02, 4.28133240e-03, 1.43844989e-03, 4.83293024e-04,
       1.62377674e-04, 5.45559478e-05, 1.83298071e-05, 6.15848211e-06,
       2.06913808e-06, 6.95192796e-07, 2.33572147e-07, 7.84759970e-08,
       2.63665090e-08, 8.85866790e-09, 2.97635144e-09, 1.00000000e-09])}],
             refit=False, scoring=['precision'], verbose=1)

In [8]:
gridcv_results = pd.DataFrame(gridcv.cv_results_)

In [9]:
best_param = gridcv_results[gridcv_results.rank_test_precision == 1]['params'].values[0]
print(f'Best Naive Bayes parameters after Grid Search is {best_param}.')

Best Naive Bayes parameters after Grid Search is {'var_smoothing': 1.0}.


In [10]:
# save the object to a file
with open(save_dir+'gridsearch_results_nb.pkl', 'wb') as f:
    pickle.dump(gridcv, f)